In [ ]:
import pandas as pd
import numpy as np
import ta

# 1. Configuración
TICKER = "ETH-USD"
INTERVALO = "1d" 
nombre_archivo_raw = f"ethereum_{INTERVALO}_raw.csv"

# 1. Cargar el CSV crudo
df = pd.read_csv(f"raw_data/{nombre_archivo_raw}", index_col=0, parse_dates=True)

# --- A. MOMENTUM: RSI (Relative Strength Index) ---
# RSI > 70 indica "Sobrecompra" -> Posible señal de Short
rsi_indicator = ta.momentum.RSIIndicator(close=df['Close'], window=14)
df['RSI'] = rsi_indicator.rsi()

# --- B. VOLATILIDAD: Bollinger Bands ---
# Si Close > BB_High, el precio está estadísticamente "caro"
bb_indicator = ta.volatility.BollingerBands(close=df['Close'], window=20, window_dev=2)
df['BB_High'] = bb_indicator.bollinger_hband()
df['BB_Low'] = bb_indicator.bollinger_lband()
df['BB_Mid'] = bb_indicator.bollinger_mavg() # Es la media móvil simple (SMA 20)
# Creamos una señal binaria: 1 si el precio rompió la banda superior (High alert for short)
df['BB_Overbought'] = df.apply(lambda x: 1 if x['Close'] > x['BB_High'] else 0, axis=1)

# --- C. TENDENCIA: MACD (Moving Average Convergence Divergence) ---
# Nos interesa cuando el histograma se vuelve negativo (cruce bajista)
macd_indicator = ta.trend.MACD(close=df['Close'])
df['MACD_Line'] = macd_indicator.macd()
df['MACD_Signal'] = macd_indicator.macd_signal()
df['MACD_Hist'] = macd_indicator.macd_diff() # El histograma muestra la fuerza de la tendencia

# --- D. GESTIÓN DE RIESGO: ATR (Average True Range) ---
# El LLM usará esto para sugerir el Stop Loss (ej: Stop Loss = Precio + 2 * ATR)
atr_indicator = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14)
df['ATR'] = atr_indicator.average_true_range()

# 2. Limpieza final (Borrar filas con NaN generados por las medias móviles iniciales)
df_final = df.dropna()

# 3. Guardar Dataset Procesado
nombre_archivo_raw = f"ethereum_{INTERVALO}_indicators_raw.csv"
df_final.to_csv(f"raw_data/{nombre_archivo_raw}")

print(f"✅ Dataset con indicadores guardado en: {nombre_archivo_raw}")
print("Columnas generadas:", df_final.columns.tolist())


✅ Dataset con indicadores guardado en: ethereum_1d_indicators.csv
Columnas generadas: ['Close', 'High', 'Low', 'Open', 'Volume', 'RSI', 'BB_High', 'BB_Low', 'BB_Mid', 'BB_Overbought', 'MACD_Line', 'MACD_Signal', 'MACD_Hist', 'ATR']
